<a href="https://colab.research.google.com/github/andohyung/MS_Project/blob/main/%EB%85%BC%EB%AC%B8%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%B6%94%EC%B6%9C_TFIDF%ED%82%A4%EC%9B%8C%EB%93%9C%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

한국어 키워드

In [2]:
pip install numpy scikit-learn konlpy PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 38.1 MB/s eta 0:00:00


In [4]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import PyPDF2

# KONLPY 설치 코드 (주석 처리하여 참고용으로 제공)
# !pip install konlpy

# PDF 파일에서 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# PDF 파일 경로 설정
pdf_path = '/content/drive/MyDrive/멋사_실전프로젝트1/프로젝트1_입력_논문리스트/일일 착용 콘택트렌즈의 연속 착용에 따른 세균 오염.pdf'

# PDF 파일에서 텍스트 추출
korean_text = extract_text_from_pdf(pdf_path)

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    return text

# 형태소 분석 및 명사 추출 함수
def extract_nouns(text):
    okt = Okt()
    nouns = okt.nouns(text)
    return [' '.join(nouns)]

# 텍스트 전처리
preprocessed_text = preprocess_text(korean_text)

# 명사만 추출
nouns_text = extract_nouns(preprocessed_text)

# TF-IDF 벡터라이저 초기화 및 키워드 추출 (ngram_range=(1, 2)로 설정하여 unigram과 bigram을 모두 고려)
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(nouns_text)

# TF-IDF 결과를 가져와서 높은 순으로 키워드 5개 추출
feature_array = np.array(vectorizer.get_feature_names_out())
tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]

# 상위 5개 키워드 출력
top_n = 10
top_keywords = feature_array[tfidf_sorting][:top_n]
print("Top 5 Keywords:", top_keywords)



ValueError: empty vocabulary; perhaps the documents only contain stop words

영어-한국어 키워드 추출

In [7]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import PyPDF2

# KONLPY 설치 코드 (주석 처리하여 참고용으로 제공)
# !pip install konlpy

# PDF 파일에서 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# PDF 파일 경로 설정
pdf_path = '/content/drive/MyDrive/멋사_실전프로젝트1/프로젝트1_입력_논문리스트/bart_paper.pdf'

# PDF 파일에서 텍스트 추출
korean_text = extract_text_from_pdf(pdf_path)

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    return text

# 형태소 분석 및 명사 추출 함수
def extract_nouns_and_english(text):
    okt = Okt()
    nouns = okt.nouns(text)
    # 영어 단어도 포함하기 위해 정규 표현식을 사용해 영어 단어 추출
    english_words = re.findall(r'[a-zA-Z]+', text)
    return [' '.join(nouns + english_words)]

# 텍스트 전처리
preprocessed_text = preprocess_text(korean_text)

# 명사 및 영어 단어 추출
nouns_text = extract_nouns_and_english(preprocessed_text)

# TF-IDF 벡터라이저 초기화 및 키워드 추출 (ngram_range=(1, 2)로 설정하여 unigram과 bigram을 모두 고려)
# 불용어(stop words) 제거를 위해 stop_words 파라미터 설정
stop_words = ['of', 'and', 'the', 'in', 'to', 'for', 'on', 'with', 'at', 'by', 'an', 'be', 'this', 'is']
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(nouns_text)

# TF-IDF 결과를 가져와서 높은 순으로 키워드 5개 추출
feature_array = np.array(vectorizer.get_feature_names_out())
tfidf_scores = tfidf_matrix.toarray().flatten()

# TF-IDF 가중치를 기준으로 정렬
tfidf_sorting = np.argsort(tfidf_scores)[::-1]

# 중복된 키워드 제거 및 상위 5개 키워드 추출
seen_keywords = set()
top_keywords = []
for idx in tfidf_sorting:
    keyword = feature_array[idx]
    if keyword not in seen_keywords:
        top_keywords.append(keyword)
        seen_keywords.add(keyword)
    if len(top_keywords) >= 10:
        break

# 상위 5개 키워드 출력
print("Top 5 Keywords:", top_keywords)

# 결과 예시 출력
top_keywords

Top 5 Keywords: ['bart', 'we', 'et al', 'et', 'al', 'model', 'are', 'tasks', 'pretraining', 'arxiv']


['bart',
 'we',
 'et al',
 'et',
 'al',
 'model',
 'are',
 'tasks',
 'pretraining',
 'arxiv']

영어 전용 키워드 추출

In [11]:
# 필요 패키지 설치
!pip install pymupdf scikit-learn

import fitz  # PyMuPDF
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re

# PDF 파일 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page_num in range(len(doc)):
            page = doc[page_num]
            text += page.get_text()
    return text

# 전처리 함수 (특수 문자 제거 및 소문자 변환)
def preprocess_text(text):
    # 불필요한 특수 문자와 숫자를 제거하고 소문자로 변환합니다.
    text = re.sub(r'\W+', ' ', text)
    return text.lower()

# TF-IDF를 통한 키워드 추출 함수
def extract_keywords_tfidf(text, top_n=10):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names_out()

    # TF-IDF 값을 배열로 변환하여 가장 높은 값의 인덱스 추출
    tfidf_scores = tfidf_matrix.toarray()[0]
    top_indices = np.argsort(tfidf_scores)[-top_n:]

    # 상위 키워드 및 점수 추출
    keywords = [(feature_names[i], tfidf_scores[i]) for i in top_indices]
    return sorted(keywords, key=lambda x: -x[1])  # 점수 내림차순 정렬

# 파일 경로 입력
pdf_path = '/content/drive/MyDrive/멋사_실전프로젝트1/프로젝트1_입력_논문리스트/bart_paper.pdf'  # 여기에 PDF 파일 경로를 넣으세요

# PDF 텍스트 추출
raw_text = extract_text_from_pdf(pdf_path)
processed_text = preprocess_text(raw_text)

# 키워드 추출
top_keywords = extract_keywords_tfidf(processed_text, top_n=20)

# 결과 출력
print("Top Keywords (TF-IDF):")
for keyword, score in top_keywords:
    print(f"{keyword}: {score}")


Top Keywords (TF-IDF):
bart: 0.3725307022304963
al: 0.2793980266728722
et: 0.2793980266728722
model: 0.25510254609262245
2019: 0.24295480580249756
pre: 0.21056083169549789
training: 0.17411761082512325
tasks: 0.16601911729837335
arxiv: 0.16196987053499837
language: 0.15792062377162341
decoder: 0.13767438995474862
encoder: 0.13767438995474862
sequence: 0.12957589642799872
text: 0.12552664966462374
models: 0.11337890937449886
bert: 0.10528041584774894
generation: 0.10528041584774894
token: 0.09718192232099902
performance: 0.09313267555762407
tokens: 0.0890834287942491
